In [98]:
from mnist import MNIST
import numpy as np
from scipy.special import expit
import time


In [99]:
def load_mnist_data():

    mndata = MNIST('')
    mndata.gz = True
    
    images, labels = mndata.load_training()
    
    test_images, test_labels = mndata.load_testing()

    return np.array(images), np.array(labels), np.array(test_images), np.array(test_labels)

In [100]:
Train_input,Train_label,Test_input,Test_label= load_mnist_data()

In [131]:
class DigitRecognition:

    
    
    def initiliaze_weights(self,hiddenLayerNum,learningCoef):
        self.Hidden_layer_weight = np.random.uniform(-0.01, 0.01, size=(hiddenLayerNum, 784))
        self.Output_layer_weight = np.random.uniform(-0.01, 0.01, size=(10, hiddenLayerNum))
        self.learningCoef = learningCoef
        self.hiddenLayerNum = hiddenLayerNum
        return

    def train(self,images,labels,batch_size,epochs):
        startTime = time.time()
        
        trainingError = 0.0
        
        for epoch in range(epochs):
            cumulativeError = 0.0
            for i in range(batch_size):
                data = np.reshape(images[i], (784, 1)) / 255
                error = self.backpropagation_case1(data, labels[i])
                cumulativeError += np.sum(error**2 * 0.5)
            
            epochErrorMean = cumulativeError / batch_size
            trainingError += epochErrorMean
            print("Epoch Completed: ", epoch + 1,"\nMean Squares Error of Epoch: " , epochErrorMean)
            
        trainingErrorMean = trainingError / 50
        stopTime = time.time()
        print("Training Completed!\nCPU Time: ", stopTime-startTime, " seconds","\nOverall Error:",trainingErrorMean)
        
    def train2(self,images,labels,batch_size,epochs):
        startTime = time.time()
        
        trainingError = 0.0
        
        for epoch in range(epochs):
            cumulativeError = 0.0
            for i in range(batch_size):
                data = np.reshape(images[i], (784, 1)) / 255
                error = self.backpropagation_case2(data, labels[i])
                cumulativeError += np.sum(error**2 * 0.5)
            
            epochErrorMean = cumulativeError / batch_size
            trainingError += epochErrorMean
            print("Epoch Completed: ", epoch + 1,"\nMean Squares Error of Epoch: " , epochErrorMean)
            
        trainingErrorMean = trainingError / 50
        stopTime = time.time()
        print("Training Completed!\nCPU Time: ", stopTime-startTime, " seconds","\nOverall Error:",trainingErrorMean)
        

    def forward_pass_case1(self,image):
        # Forward Hidden Layer
        z_1 = np.dot(self.Hidden_layer_weight,image) 
        o_1 = self.Tanh_activation(z_1) # For Case 1
        #o1 = self.activation(v1,self.Relu) # For Case 2
        
        # Forward Output Layer
        z_2 = np.dot(self.Output_layer_weight,o_1) 
        o_2 = self.Tanh_activation(z_2) # For Case 1
        #o2 = self.activation(v2,self.Relu) # For Case 2

        return o_1,o_2
    
    def forward_pass_case2(self,image):
        # Forward Hidden Layer
        z_1 = np.dot(self.Hidden_layer_weight,image) 
        o_1 = self.relu_activation(z_1) # For Case 1
        #o1 = self.activation(v1,self.Relu) # For Case 2
        
        # Forward Output Layer
        z_2 = np.dot(self.Output_layer_weight,o_1) 
        o_2 = self.sigmoid_activation(z_2) # For Case 1
        #o2 = self.activation(v2,self.Relu) # For Case 2

        return o_1,o_2
    

    def relu_activation(self,x):
        
        return np.maximum(x, 0)
    
    def relu_activation_derivative(self,x):
        x[x>=0]=1
        x[x<0]=0
        return x
    

    def sigmoid_activation(self,x):
        y = expit(x)
        return y
    
    def sigmoid_activation_derivative(self,x):
        return x-x*x
    
    def Tanh_activation(self,x):
        return np.tanh(x)
        
    def Tanh_activation_derivative(self,x):
        return (1/2)*(1-x*x)

    
    def desired_tanh(self,label):
        d = -np.ones((10,1))
        d[label] = 1
        return d
    
    def desired_sigmoid(self,label):
        d = np.zeros((10,1))
        d[label] = 1
        return d
    
    def error_cal(self,prediction,desired):
        e = desired - prediction
        return e
    
    def calculate_output_gradient_case1(self,error,o_2):
        derivative =  np.eye(10) * self.Tanh_activation_derivative(o_2)
        gradient = np.dot(derivative,error)
        
        return np.reshape(gradient,(10,1))
    
    def calculate_output_gradient_case2(self,error,o_2):
        derivative =  np.eye(10) * self.sigmoid_activation_derivative(o_2)
        gradient = np.dot(derivative,error)
        
        return np.reshape(gradient,(10,1))
    
   
    def calculate_first_gradient_case1(self,o_1,output_gradient):
        derivative = np.eye(self.hiddenLayerNum) * self.Tanh_activation_derivative(o_1)

        gradient = np.dot( np.dot( derivative ,np.transpose(self.Output_layer_weight)), output_gradient)

        return np.reshape(gradient,(self.hiddenLayerNum,1))
    
    
    def calculate_first_gradient_case2(self,o_1,output_gradient):
        derivative = np.eye(self.hiddenLayerNum) * self.relu_activation_derivative(o_1)

        gradient = np.dot( np.dot( derivative ,np.transpose(self.Output_layer_weight)), output_gradient)

        return np.reshape(gradient,(self.hiddenLayerNum,1))
    
    
    
    def evaluate_accuracy(self,images,labels):
        cumulativeError = 0.0
        numOfMissClassification = 0
        
        for i in range(10000):
            testData = np.reshape(images[i], (784, 1)) / 255
            o1, output = self.forward_pass_case1(testData)
                
            d = self.desired_tanh(labels[i])
            error = self.error_cal(output,d)
            cumulativeError += np.sum(error**2 * 0.5)
            
            predicted = np.argmax(output)
            
            if labels[i] != predicted:
                numOfMissClassification += 1
        
        misclasificationPercentage = numOfMissClassification / 10000 * 100
                
        testSetErrorMean = cumulativeError / 10000
        
        print("Number Of Misclasification:",numOfMissClassification,"\nError Percentage: %",misclasificationPercentage,"\nTest Data Mean Square Error:",testSetErrorMean)
     
    
    def evaluate_accuracy2(self,images,labels):
        cumulativeError = 0.0
        numOfMissClassification = 0
        
        for i in range(10000):
            testData = np.reshape(images[i], (784, 1)) / 255
            o1, output = self.forward_pass_case2(testData)
                
            d = self.desired_sigmoid(labels[i])
            error = self.error_cal(output,d)
            cumulativeError += np.sum(error**2 * 0.5)
            
            predicted = np.argmax(output)
            
            if labels[i] != predicted:
                numOfMissClassification += 1
        
        misclasificationPercentage = numOfMissClassification / 10000 * 100
                
        testSetErrorMean = cumulativeError / 10000
        
        print("Number Of Misclasification:",numOfMissClassification,"\nError Percentage: %",misclasificationPercentage,"\nTest Data Mean Square Error:",testSetErrorMean)
     
    

    def backpropagation_case1(self,inputs, labels):
        o_1,o_2 = self.forward_pass_case1(inputs)
        desired = self.desired_tanh(labels)
        error = self.error_cal(o_2,desired)
        output_gradients = self.calculate_output_gradient_case1(error,o_2)
        self.Output_layer_weight = self.Output_layer_weight + self.learningCoef * np.dot(output_gradients, o_1.T)
        input_gradients = self.calculate_first_gradient_case1(o_1,output_gradients)
        self.Hidden_layer_weight = self.Hidden_layer_weight + self.learningCoef * np.dot(input_gradients, inputs.T)
        
        return error
    
    def backpropagation_case2(self,inputs, labels):
        o_1,o_2 = self.forward_pass_case2(inputs)
        desired = self.desired_sigmoid(labels)
        error = self.error_cal(o_2,desired)
        output_gradients = self.calculate_output_gradient_case2(error,o_2)
        self.Output_layer_weight = self.Output_layer_weight + self.learningCoef * np.dot(output_gradients, o_1.T)
        input_gradients = self.calculate_first_gradient_case2(o_1,output_gradients)
        self.Hidden_layer_weight = self.Hidden_layer_weight + self.learningCoef * np.dot(input_gradients, inputs.T)
        
        return error

In [132]:
def Train_and_Test(hidden_layer, learning_coef, batch_size, epochs, Case_selection):
    Case2=DigitRecognition()
    Case1=DigitRecognition()
    if Case_selection == 1:
        Case1.initiliaze_weights(hidden_layer, learning_coef)
        Case1.train(Train_input,Train_label,batch_size,epochs)
        Case1.evaluate_accuracy(Test_input,Test_label)
    if Case_selection == 2:
        Case2.initiliaze_weights(hidden_layer, learning_coef)
        Case2.train2(Train_input,Train_label,batch_size,epochs)
        Case2.evaluate_accuracy2(Test_input,Test_label)


In [118]:
Case1=DigitRecognition()

In [119]:
Case1.initiliaze_weights(300, 0.01)

In [112]:
Case1.train(Train_input,Train_label,1250,5)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.861314248942573
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.2867377332351853
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.9056150183534318
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.7044632935931532
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.5680854423692993
Training Completed!
CPU Time:  15.335951089859009  seconds 
Overall Error: 0.10652431472987285


In [63]:
Case1.evaluate_accuracy(Test_input,Test_label)

Number Of Misclasification: 1780 
Error Percentage: % 17.8 
Test Data Mean Square Error: 0.6511027215390969


In [68]:
Case2=DigitRecognition()
Case2.initiliaze_weights(300, 0.01)
Case2.train_2(Train_input,Train_label,1250,5)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.5480682184636686
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.4560525350403728
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.4383357361846098
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.4016941353074904
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.3543992697162048
Training Completed!
CPU Time:  11.297040700912476  seconds 
Overall Error: 0.043970997894246924


In [69]:
Case2.evaluate_accuracy(Test_input,Test_label)

Number Of Misclasification: 5956 
Error Percentage: % 59.56 
Test Data Mean Square Error: 1.36887526626679


In [72]:
Train_and_Evaluate(300, 0.01, 1250, 5, 2)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.867011754096712
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.2913323143024031
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.905625289450323
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.7081093253443134
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.5720199785577205
Training Completed!
CPU Time:  11.721179008483887  seconds 
Overall Error: 0.10688197323502945
Number Of Misclasification: 1778 
Error Percentage: % 17.78 
Test Data Mean Square Error: 0.639101854754517


In [74]:
Train_and_Test(300, 0.01, 1250, 50, 1)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.8652554322115888
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.2944029627713547
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.9113317555560608
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.7120348346878919
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.5876567520517042
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.4826192250238314
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.42101861599393603
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.3799904810327406
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.3485593186757849
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.32307010951321197
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.3015611452508781
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.2827401685580495
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.26600906982934486
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.25092588684394146
Epoch Compl

In [75]:
Train_and_Test(500, 0.01, 1250, 50, 1)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.7938685988527272
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.146432730297519
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.8173995095639583
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.6685722508088096
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.5459282958464383
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.4620873679805046
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.41014934914374507
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.3734055546366177
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.3449321401085371
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.32191752409670793
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.3022683102612021
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.2846783119589092
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.2686079344599692
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.2539182625284798
Epoch Complete

In [76]:
Train_and_Test(1000, 0.01, 1250, 50, 1)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.6632768660891666
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.9936437281525037
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.7345112144082458
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.5933099979801038
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.5007710956321354
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.4422683309731913
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.39909980354064284
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.36627306101325957
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.34033301614075673
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.3194864622097597
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.3017030703143523
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.2860134121693316
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.2717929808658259
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.2587751920915917
Epoch Comple

In [77]:
Train_and_Test(300, 0.05, 1250, 50, 1)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.623127854584984
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.3060152449969844
Epoch Completed:  3 
Mean Squares Error of Epoch:  1.0232454559463768
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.8592197396323245
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.7742501127555527
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.7286031805465674
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.7198750528592142
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.6782373338675467
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.6635032212355634
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.5631046855472431
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.48902620391300833
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.5045056265736947
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.47844280160672414
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.4349009685514951
Epoch Complete

In [78]:
Train_and_Test(500, 0.05, 1250, 50, 1)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.6242867648687063
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.3493200826449694
Epoch Completed:  3 
Mean Squares Error of Epoch:  1.13783520594723
Epoch Completed:  4 
Mean Squares Error of Epoch:  1.1907072197526554
Epoch Completed:  5 
Mean Squares Error of Epoch:  1.0836308845245368
Epoch Completed:  6 
Mean Squares Error of Epoch:  1.0228812520435826
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.9883268700965795
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.9773448578239448
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.9242222653874004
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.9376948727085761
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.9426772718644834
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.9471516772760223
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.9321710125021917
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.9335856337899433
Epoch Completed: 

In [79]:
Train_and_Test(1000, 0.05, 1250, 50, 1)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.6420915243069711
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.4575851289467185
Epoch Completed:  3 
Mean Squares Error of Epoch:  1.4288883876961764
Epoch Completed:  4 
Mean Squares Error of Epoch:  1.4060439081756713
Epoch Completed:  5 
Mean Squares Error of Epoch:  1.3571108320566283
Epoch Completed:  6 
Mean Squares Error of Epoch:  1.349255031462023
Epoch Completed:  7 
Mean Squares Error of Epoch:  1.3361028153127845
Epoch Completed:  8 
Mean Squares Error of Epoch:  1.3232302088877348
Epoch Completed:  9 
Mean Squares Error of Epoch:  1.3436763974370807
Epoch Completed:  10 
Mean Squares Error of Epoch:  1.3701406358029162
Epoch Completed:  11 
Mean Squares Error of Epoch:  1.2988257086544754
Epoch Completed:  12 
Mean Squares Error of Epoch:  1.301946841972789
Epoch Completed:  13 
Mean Squares Error of Epoch:  1.2735249871336012
Epoch Completed:  14 
Mean Squares Error of Epoch:  1.260426021187935
Epoch Completed:  

In [80]:
Train_and_Test(300, 0.09, 1250, 50, 1)

Epoch Completed:  1 
Mean Squares Error of Epoch:  2.0583807870752917
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.7588267637295811
Epoch Completed:  3 
Mean Squares Error of Epoch:  1.6690714856125883
Epoch Completed:  4 
Mean Squares Error of Epoch:  1.6629260197232207
Epoch Completed:  5 
Mean Squares Error of Epoch:  1.6490393486881028
Epoch Completed:  6 
Mean Squares Error of Epoch:  1.658257124444216
Epoch Completed:  7 
Mean Squares Error of Epoch:  1.6534177816516655
Epoch Completed:  8 
Mean Squares Error of Epoch:  1.6512109434402071
Epoch Completed:  9 
Mean Squares Error of Epoch:  1.6344163685298365
Epoch Completed:  10 
Mean Squares Error of Epoch:  1.6198649171939963
Epoch Completed:  11 
Mean Squares Error of Epoch:  1.6271145222243584
Epoch Completed:  12 
Mean Squares Error of Epoch:  1.611288479362464
Epoch Completed:  13 
Mean Squares Error of Epoch:  1.6081222867667182
Epoch Completed:  14 
Mean Squares Error of Epoch:  1.610079561538094
Epoch Completed:  

In [81]:
Train_and_Test(500, 0.09, 1250, 50, 1)

Epoch Completed:  1 
Mean Squares Error of Epoch:  2.199973109839886
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.8224061828292197
Epoch Completed:  3 
Mean Squares Error of Epoch:  2.0155030210905185
Epoch Completed:  4 
Mean Squares Error of Epoch:  1.7207395536617873
Epoch Completed:  5 
Mean Squares Error of Epoch:  1.718592425206966
Epoch Completed:  6 
Mean Squares Error of Epoch:  1.6679023981149523
Epoch Completed:  7 
Mean Squares Error of Epoch:  1.6809000256189197
Epoch Completed:  8 
Mean Squares Error of Epoch:  1.6841825072101437
Epoch Completed:  9 
Mean Squares Error of Epoch:  1.6936122882868687
Epoch Completed:  10 
Mean Squares Error of Epoch:  1.6481539268845422
Epoch Completed:  11 
Mean Squares Error of Epoch:  1.6481034611775291
Epoch Completed:  12 
Mean Squares Error of Epoch:  1.6314798890052717
Epoch Completed:  13 
Mean Squares Error of Epoch:  1.6169355983346074
Epoch Completed:  14 
Mean Squares Error of Epoch:  1.6702692820172829
Epoch Completed: 

In [82]:
Train_and_Test(1000, 0.09, 1250, 50, 1)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.9802162544735133
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.9943972490462205
Epoch Completed:  3 
Mean Squares Error of Epoch:  1.9605916092632332
Epoch Completed:  4 
Mean Squares Error of Epoch:  1.8500539122161426
Epoch Completed:  5 
Mean Squares Error of Epoch:  1.802272137072397
Epoch Completed:  6 
Mean Squares Error of Epoch:  1.7715644641527597
Epoch Completed:  7 
Mean Squares Error of Epoch:  1.7262877607960159
Epoch Completed:  8 
Mean Squares Error of Epoch:  1.7124151749243959
Epoch Completed:  9 
Mean Squares Error of Epoch:  1.7582550344383063
Epoch Completed:  10 
Mean Squares Error of Epoch:  1.7158676768882437
Epoch Completed:  11 
Mean Squares Error of Epoch:  1.654259235271587
Epoch Completed:  12 
Mean Squares Error of Epoch:  1.648421912447528
Epoch Completed:  13 
Mean Squares Error of Epoch:  1.6283598723951795
Epoch Completed:  14 
Mean Squares Error of Epoch:  1.6349537962486262
Epoch Completed:  

In [83]:
Train_and_Test(300, 0.01, 1250, 50, 2)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.8659979114954415
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.2954006468820525
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.9028843120938657
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.7045575922231422
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.5682103772458559
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.4739394366407583
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.4179531708617389
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.3788668693385471
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.34833101013952716
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.32344034678623124
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.3021663193375975
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.2833077433736758
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.26635941998806373
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.2509531333352212
Epoch Comple

In [84]:
Train_and_Test(500, 0.01, 1250, 50, 2)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.792889207464908
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.1583968743856423
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.8191654895932956
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.6615775017132212
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.5353869858923306
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.4558916475179571
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.4065688170542179
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.3707389905974122
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.3425034297824702
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.3197068426149872
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.30012221378595355
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.28256474680423377
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.26651600115979324
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.25190301240462354
Epoch Comple

In [85]:
Train_and_Test(1000, 0.01, 1250, 50, 2)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.6656162111751063
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.9954291938673612
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.7385121191259868
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.598017045347103
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.5028146505428428
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.4438470225612616
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.40021290768312845
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.36697037661902193
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.34108301736895574
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.3201612992101684
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.30239146576795456
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.2865833348440672
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.2721683109101021
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.25888193121127506
Epoch Compl

In [86]:
Train_and_Test(300, 0.05, 1250, 50, 2)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.5955074079488576
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.2908142099628286
Epoch Completed:  3 
Mean Squares Error of Epoch:  1.0198018932059327
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.9227498913523977
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.8502720398522879
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.7887082465050468
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.7223081774824073
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.6828836785157266
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.6372864112893957
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.6064073926616894
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.6188142608406544
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.5352616122047817
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.47820823052676
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.45282305903218484
Epoch Completed:

In [87]:
Train_and_Test(500, 0.05, 1250, 50, 2)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.6283303163577807
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.361446578092454
Epoch Completed:  3 
Mean Squares Error of Epoch:  1.2455087228566473
Epoch Completed:  4 
Mean Squares Error of Epoch:  1.121319826019916
Epoch Completed:  5 
Mean Squares Error of Epoch:  1.0860864779820822
Epoch Completed:  6 
Mean Squares Error of Epoch:  1.0328082249356803
Epoch Completed:  7 
Mean Squares Error of Epoch:  1.027891211929465
Epoch Completed:  8 
Mean Squares Error of Epoch:  1.0259448273723721
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.9574733973433597
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.9363021822021382
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.9221628329886337
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.9361158834292593
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.9395659168765831
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.902832478908625
Epoch Completed:  1

In [88]:
Train_and_Test(1000, 0.05, 1250, 50, 2)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.6663821926322133
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.4688741524550306
Epoch Completed:  3 
Mean Squares Error of Epoch:  1.4567033144362629
Epoch Completed:  4 
Mean Squares Error of Epoch:  1.359103459026999
Epoch Completed:  5 
Mean Squares Error of Epoch:  1.4440474253535382
Epoch Completed:  6 
Mean Squares Error of Epoch:  1.540466625138777
Epoch Completed:  7 
Mean Squares Error of Epoch:  1.4339461262830944
Epoch Completed:  8 
Mean Squares Error of Epoch:  1.462021749938771
Epoch Completed:  9 
Mean Squares Error of Epoch:  2.4386175508522125
Epoch Completed:  10 
Mean Squares Error of Epoch:  1.3133736177614042
Epoch Completed:  11 
Mean Squares Error of Epoch:  1.2727054711963608
Epoch Completed:  12 
Mean Squares Error of Epoch:  1.2839026793724446
Epoch Completed:  13 
Mean Squares Error of Epoch:  1.2961499604056812
Epoch Completed:  14 
Mean Squares Error of Epoch:  1.2689908767626263
Epoch Completed:  

In [89]:
Train_and_Test(300, 0.09, 1250, 50, 2)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.935149514940576
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.8770444551453893
Epoch Completed:  3 
Mean Squares Error of Epoch:  1.8977599113777126
Epoch Completed:  4 
Mean Squares Error of Epoch:  1.7713196182693551
Epoch Completed:  5 
Mean Squares Error of Epoch:  1.6763331356330564
Epoch Completed:  6 
Mean Squares Error of Epoch:  1.652805749273143
Epoch Completed:  7 
Mean Squares Error of Epoch:  1.6360198435230817
Epoch Completed:  8 
Mean Squares Error of Epoch:  1.531458015629679
Epoch Completed:  9 
Mean Squares Error of Epoch:  1.6308948487989394
Epoch Completed:  10 
Mean Squares Error of Epoch:  1.5324767908381907
Epoch Completed:  11 
Mean Squares Error of Epoch:  1.5278622930794037
Epoch Completed:  12 
Mean Squares Error of Epoch:  1.509212006237707
Epoch Completed:  13 
Mean Squares Error of Epoch:  1.5432374000338787
Epoch Completed:  14 
Mean Squares Error of Epoch:  1.619699571795301
Epoch Completed:  15

In [90]:
Train_and_Test(500, 0.09, 1250, 50, 2)

Epoch Completed:  1 
Mean Squares Error of Epoch:  2.5680396254315676
Epoch Completed:  2 
Mean Squares Error of Epoch:  3.7984671065712434
Epoch Completed:  3 
Mean Squares Error of Epoch:  5.079854947980552
Epoch Completed:  4 
Mean Squares Error of Epoch:  3.5743883326602632
Epoch Completed:  5 
Mean Squares Error of Epoch:  3.021693578783853
Epoch Completed:  6 
Mean Squares Error of Epoch:  1.7834670805509745
Epoch Completed:  7 
Mean Squares Error of Epoch:  1.6759657527578786
Epoch Completed:  8 
Mean Squares Error of Epoch:  1.631660210173167
Epoch Completed:  9 
Mean Squares Error of Epoch:  1.6612097542998148
Epoch Completed:  10 
Mean Squares Error of Epoch:  1.6638812083971475
Epoch Completed:  11 
Mean Squares Error of Epoch:  1.6781258630228297
Epoch Completed:  12 
Mean Squares Error of Epoch:  1.6555742843100276
Epoch Completed:  13 
Mean Squares Error of Epoch:  1.6286108963109838
Epoch Completed:  14 
Mean Squares Error of Epoch:  1.6173228018964494
Epoch Completed:  

In [91]:
Train_and_Test(1000, 0.09, 1250, 50, 2)

Epoch Completed:  1 
Mean Squares Error of Epoch:  2.1186636500466425
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.8258814009601232
Epoch Completed:  3 
Mean Squares Error of Epoch:  1.8430364460170738
Epoch Completed:  4 
Mean Squares Error of Epoch:  2.4741701897932513
Epoch Completed:  5 
Mean Squares Error of Epoch:  1.8172917734426857
Epoch Completed:  6 
Mean Squares Error of Epoch:  1.8014923745466032
Epoch Completed:  7 
Mean Squares Error of Epoch:  1.7928322782006512
Epoch Completed:  8 
Mean Squares Error of Epoch:  1.794259179831862
Epoch Completed:  9 
Mean Squares Error of Epoch:  1.8106106865231584
Epoch Completed:  10 
Mean Squares Error of Epoch:  1.7980916991220008
Epoch Completed:  11 
Mean Squares Error of Epoch:  1.7905071011779976
Epoch Completed:  12 
Mean Squares Error of Epoch:  1.7894839401648628
Epoch Completed:  13 
Mean Squares Error of Epoch:  1.7917318631482846
Epoch Completed:  14 
Mean Squares Error of Epoch:  1.7955489295340605
Epoch Completed:

In [92]:
print('done')

done


In [114]:
Train_and_Test(500, 0.01, 1250, 5, 1)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.7950723339573138
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.15697235828094
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.819462880406082
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.6658021819744427
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.5449917329454044
Training Completed!
CPU Time:  28.322537660598755  seconds 
Overall Error: 0.09964602975128364
Number Of Misclasification: 1788 
Error Percentage: % 17.88 
Test Data Mean Square Error: 0.6560986529037909


In [123]:
Train_and_Test(500, 0.01, 1250, 5, 2)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.537606144030697
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.4539944201047613
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.4310672687126691
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.3855506947918193
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.33441270704400256
Training Completed!
CPU Time:  26.373695611953735  seconds 
Overall Error: 0.04285262469367899
Number Of Misclasification: 5491 
Error Percentage: % 54.910000000000004 
Test Data Mean Square Error: 5.587007844398244


In [124]:
Train_and_Test(500, 0.01, 1250, 15, 2)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.5407890122434702
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.4528600343783104
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.42719065133801687
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.3786795552321435
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.3264736989861899
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.28589149580096385
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.2488175491188006
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.21574049906544387
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.19269756726831153


KeyboardInterrupt: 

In [129]:
Train_and_Test(500, 0.01, 1250, 5, 2)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.539115419340163
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.4533865858992727
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.4286135868076459
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.3798846559398529
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.32942793596420344
Training Completed!
CPU Time:  28.006203651428223  seconds 
Overall Error: 0.042608563679022764
Number Of Misclasification: 5295 
Error Percentage: % 52.949999999999996 
Test Data Mean Square Error: 0.321931195847313


In [133]:
Train_and_Test(500, 0.01, 1250, 15, 2)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.5396131457781732
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.45313585912513915
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.4282075793194059
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.38117766414966303
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.330147644735373
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.29095161528153146
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.2537909894191957
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.21988113172425888
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.19574560468514354
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.17874165739427592
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.16528188793324047
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.1532759999414349
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.14104956874030913
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.12890760160693068
Epoch C